In [1]:
import torch
import torch.nn as nn

In [2]:
features = 2 # E.G. ['breathing_rate', 'blood_pressure']
hidden_size = 3  # Hidden State size.
batch_size = 1 # E.G. One Person.
time_steps = 24 # E.G. 24 hours, capture health data ['heart_rate', 'blood_pressure'] every hour.

In [3]:
rnn = nn.RNN(input_size = features, hidden_size = hidden_size, batch_first=True)

`batch_first=True` --> Input shape will be `(batch_size, time_steps, features)`

**Ordered data can be stored as a Rank 3 tensor**

For example:

Every hour, measure `features = 2`, such as `['breathing_rate', 'blood_pressure']`.

A day (24 hours) of measurements --> Rank 2 tensor, Shape `[24, 2]`.

30 people's measurements         --> Rank 3 tensor, Shape `[30, 24, 2]`.

In [4]:
example_ordered_data = torch.rand(batch_size, time_steps, features)
print(example_ordered_data)

tensor([[[1.0000, 0.6203],
         [0.7011, 0.7608],
         [0.2176, 0.0631],
         [0.3910, 0.2742],
         [0.9379, 0.4096],
         [0.1392, 0.7074],
         [0.5251, 0.0778],
         [0.3939, 0.0132],
         [0.6146, 0.1024],
         [0.7200, 0.3159],
         [0.0527, 0.2098],
         [0.8207, 0.7207],
         [0.3945, 0.6380],
         [0.4052, 0.7231],
         [0.0221, 0.5145],
         [0.7611, 0.3566],
         [0.3306, 0.5538],
         [0.8841, 0.5581],
         [0.1950, 0.9009],
         [0.9906, 0.9472],
         [0.0075, 0.0713],
         [0.9093, 0.2202],
         [0.5476, 0.3340],
         [0.5686, 0.2868]]])


In [5]:
print(example_ordered_data.shape)

torch.Size([1, 24, 2])


In [6]:
history_hidden_states, final_hidden_state = rnn(example_ordered_data)

In [7]:
print('history_hidden_states: \n', history_hidden_states)

history_hidden_states: 
 tensor([[[ 0.5006,  0.4074,  0.1494],
         [ 0.3066,  0.2466, -0.0606],
         [ 0.2871, -0.0355,  0.2916],
         [ 0.2837, -0.0808,  0.2586],
         [ 0.4883,  0.1033,  0.1814],
         [ 0.1880, -0.0330, -0.0364],
         [ 0.4516,  0.0058,  0.3135],
         [ 0.3235, -0.2461,  0.3154],
         [ 0.4466, -0.1847,  0.3074],
         [ 0.4723, -0.1261,  0.1642],
         [ 0.2886, -0.2970,  0.1849],
         [ 0.5156,  0.1100,  0.0040],
         [ 0.3674,  0.0330, -0.0579],
         [ 0.3805,  0.1295, -0.0550],
         [ 0.2257, -0.0163,  0.0512],
         [ 0.4772,  0.1343,  0.1915],
         [ 0.2595, -0.0196,  0.0426],
         [ 0.5101,  0.2169,  0.0801],
         [ 0.1978,  0.1006, -0.1514],
         [ 0.5309,  0.4648, -0.1137],
         [ 0.2020, -0.1184,  0.2055],
         [ 0.5071,  0.0708,  0.2853],
         [ 0.3519, -0.0976,  0.1462],
         [ 0.4366, -0.0602,  0.1896]]], grad_fn=<TransposeBackward1>)


In [8]:
print('final_hidden_state: \n', final_hidden_state)

final_hidden_state: 
 tensor([[[ 0.4366, -0.0602,  0.1896]]], grad_fn=<StackBackward0>)


**For learning purposes, let's calculate the hidden state after the first time step manually.**

There will be:
1.   Input Weights.
2.   Hidden State Weights.

In [9]:
for name, _ in rnn.named_parameters():
    if 'weight' or 'bias' in name:
        print(name)

weight_ih_l0
weight_hh_l0
bias_ih_l0
bias_hh_l0


In [10]:
input_weights = rnn.weight_ih_l0
input_biases = rnn.bias_ih_l0

hidden_state_weights = rnn.weight_hh_l0
hidden_state_biases = rnn.bias_hh_l0

**Current Hidden State =  Activation Function ( (Hidden State Weights * Past Hidden State) + (Input Weights * Current Input) )**

In [11]:
# Initial hidden state will be zeros.
hidden_state_custom = torch.zeros((batch_size, hidden_size))
print('Initial hidden state', hidden_state_custom.detach())

# (Hidden State Weights * Past Hidden State)
hidden_to_hidden = torch.matmul(hidden_state_custom, torch.t(hidden_state_weights)) + hidden_state_biases

# (Input Weights * Current Input)
current_input = example_ordered_data[:, 0]
input_to_hidden = torch.matmul(current_input, torch.t(input_weights)) + input_biases

# Current Hidden State = Activation Function ( (Hidden State Weights * Past Hidden State) + (Input Weights * Current Input) )
hidden_state_custom = torch.tanh(hidden_to_hidden + input_to_hidden)

print('hidden_state_custom after the first time step :', hidden_state_custom.detach())
print('Hidden state after the first time step:', history_hidden_states[:, 0].detach())

Initial hidden state tensor([[0., 0., 0.]])
hidden_state_custom after the first time step : tensor([[0.5006, 0.4074, 0.1494]])
Hidden state after the first time step: tensor([[0.5006, 0.4074, 0.1494]])
